In [1]:
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate,GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
import nltk
import pandas as pd
from nltk import CRFTagger
from sklearn.model_selection import train_test_split
from collections import Counter
import pycrfsuite
from nltk.tag import CRFTagger
print("import done")

import done


In [3]:
# buat satu file .txt yang berisi keseluruhan nama dokumen
# buat forloop untuk membaca dokumen csv
filesnames = os.listdir("../Dataset/Raw/")
filesnames = [f for f in filesnames if (f.lower().endswith(".csv"))]
print(filesnames)

['templated_posneg_dataset_komentar_instagram_cyberbullying.csv', 'templated_posneg_dataset_tweet_sentiment_cellular_service_provider.csv', 'templated_posneg_dataset_tweet_sentimen_tayangan_tv.csv', 'templated_posneg_Dataset_C_HT_4000.csv', 'templated_posneg_dataset_tweet_sentiment_opini_film.csv', 'templated_posneg_dataset_tweet_sentiment_pilkada_DKI_2017.csv', 'templated_posneg_id-apps-review-sentimentanalysis.csv', 'templated_posneg_id-movie-review-sentimentanalysis.csv', 'templated_posneg_KicauanBerlabel.csv', 'templated_posnegnet_tweets_tagged_preprocessed.csv', 'templated_posnegnet_tweetcleannew600-only.csv', 'templated_posneg_raw_data_sentiment2.csv', 'templated_posneg_hotel.csv']


In [4]:
def dict_inject ():
    dict = {}
    fileinsetnegative = open("../Machine Learning/featureExtraction/lexicon/lexicon-source/InSet/negative.tsv")
    fileinsetpositive = open("../Machine Learning/featureExtraction/lexicon/lexicon-source/InSet/negative.tsv")
    counter = 0;
    counter_neg = 0;
    for line in fileinsetnegative:
        if counter == 0:
            counter += 1
            continue
            
        key, value = line.split("\t")
        dict[key] = float(value)
    for line in fileinsetpositive:
        if counter_neg == 0:
            counter_neg += 1
            continue
        key, value = line.split("\t")
        dict[key] = float(value)
    return(dict);
dict = dict_inject()

In [5]:
def adjective_only(token, posneg):
    word_counter = 0;
    score = 0;
    #print('============================= adj only ==============================')
    for word in token:
        if word in dict:
            word_score = dict[word]
            word_counter += 1
            score += word_score

    if posneg == "posneg":
        if word_counter == 0:
            return 1;
        if score <=0 :
            return 0;
        else :
            return 1;
    elif posneg == "posnegnet":
        if word_counter == 0:
            return 0;
        if score <0 :
            return -1;
        elif score == 0:
            return 0;
        else :
            return 1;



In [6]:
def repetitif(token,posneg):
    used_word_dict = {};
    word_counter = 0;
    score = 0;
    #print('============================= adj only ==============================')
    #print(token)

    token = nltk.word_tokenize(token)

    for word in token:
        if word in dict:
            word_score = dict[word]
            word_counter += 1
            if word in used_word_dict:
                used_word_dict[word] += 1
                #print(word_score)
                word_score /= used_word_dict[word]
                #print(word_score)
            else:
                used_word_dict[word] = 1
            score += word_score

            #print(word)
            #print(used_word_dict[word])
            #print(score)
            #print("--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- test -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")

    if posneg == "posneg":
        if word_counter == 0:
            return 1;
        if score <=0 :
            return 0;
        else :
            return 1;
    elif posneg == "posnegnet":
        if word_counter == 0:
            return 0;
        if score <0 :
            return -1;
        elif score == 0:
            return 0;
        else :
            return 1;

In [7]:
def negation(token, posneg):
    word_counter = 0;
    score = 0;
    negation_mark = False
    #print('============================= adj only ==============================')
    token = nltk.word_tokenize(token)

    for word in token:
        if word in neg:
            negation_mark = True
        elif word in dict:
            word_score = dict[word]
            word_counter += 1
            #print(word_score)
            if negation_mark :
                word_score = word_score * -1
            #print(word_score)
            score += word_score
        elif word in [".","!",";", "(" , ")" , ":" , "?"]:
            negation_mark = False

    if posneg == "posneg":
        if word_counter == 0:
            return 1;
        if score <=0 :
            return 0;
        else :
            return 1;
    elif posneg == "posnegnet":
        if word_counter == 0:
            return 0;
        if score <0 :
            return -1;
        elif score == 0:
            return 0;
        else :
            return 1;



In [8]:
def accuration(data):
    acc = 0
    counter = 0
    for index, row in data.iterrows():
#        print(row['label'], row['newlabel'])
        #print(row['label'] , row['newlabel'])
        if str(row['label']) == str(row['newlabel']):
            acc += 1
            counter+=1
        else :
            acc += 0
            counter+=1

    print("acc" , acc)
    print("counter" , counter)
    return acc/counter;

In [9]:
def fone(data):
    score = 0
    score = f1_score(data['label'].astype(str), data['newlabel'].astype(str), average='micro')
    return score

In [10]:
def fone_macro(data):
    score = 0
    score = f1_score(data['label'].astype(str), data['newlabel'].astype(str), average='macro')
    return score

In [11]:
neg = [ "tidak", "bukan", "belum","jangan"]
intensfront = ["agak",
               "amat",
               "sangat",
               "paling",
               "rada",
               "sungguh",
               "terlalu",
               "sedikit"]
intensback = ["nian",
              "sekali",
              "banget"]
from nltk.tag import CRFTagger
ct = CRFTagger()
ct.set_model_file('../Machine Learning/featureExtraction/pos/all_indo_man_tag_corpus_model.crf.tagger')

def tagging(text):
    token = nltk.word_tokenize(text)
    adjp = []
    for word, post in ct.tag(token):
#        print(word, "->", post)
        
        if post == "JJ":
            adjp.append(word)
    return adjp
#sent = ["acara", "padu", "suara", "ini" ,",","sangat", "bagus", "dan", "tarik"]
#print(tagging("acara padu suara ini sangat  / > < { ] asdasd dunno i dont know , ; ! ? aaa bagus."))

In [12]:
#print(nltk.pos_tag("saya pergi ke pasar".split()))

In [13]:
f = open("result.txt", "w")
counter = 0
lst_acc_rep =[]
lst_f1_rep = []
lst_acc_neg =[]
lst_f1_neg = []
lst_acc_adj =[]
lst_f1_adj = []
for fs in filesnames:
    f.write(fs + "\n")
    if fs == "templated_posnegnet_tweetcleannew600-only.csv":
        banyak_label = "posnegnet"
        
    elif fs  == 'templated_posnegnet_tweets_tagged_preprocessed.csv':
        banyak_label = "posnegnet"
    else :
        banyak_label = "posneg"
        
    print("\n\n\n\n===================================================================")

    print(fs)
    print(banyak_label)
    colnames = ["teks", "label"]
    df = pd.read_csv("../Dataset/AL-P/" + fs ,sep=";", engine="python", names=colnames, header=0)
    df.dropna(subset = ["teks"],inplace=True)

    df['newlabel'] = df['teks'].apply(lambda x: repetitif(x, banyak_label))

    f.write("repetitif" + "\n")
    res_acc = accuration(df)
    print(res_acc)
    print("===================================================================")
    #res_f1 = fone(df)
    #print(res_f1)
    #print("===================================================================")
    res_f1_macro = fone_macro(df)
    print(res_f1_macro)
    lst_acc_rep.append(res_acc)
    lst_f1_rep.append(res_f1_macro)
    print("===================================================================")
    f.write("acc" + "\n"+ str(res_acc)  + "\n"+ "f1_macro" +  "\n"+ str(res_f1_macro) + "\n");

    df['adj'] = df['teks'].apply(lambda x: tagging(str(x)))

    df['newlabel'] = df['adj'].apply(lambda x: adjective_only(x,banyak_label))
    f.write("adj_only" + "\n")

    res_acc = accuration(df)
    print(res_acc)
    print("===================================================================")
    #res_f1 = fone(df)
    #print(res_f1)
    #print("===================================================================")
    res_f1_macro = fone_macro(df)
    print(res_f1_macro)
    print("===================================================================")
    lst_acc_adj.append(res_acc)
    lst_f1_adj.append(res_f1_macro)
    f.write("acc" + "\n"+ str(res_acc) + "\n" +  "f1_macro" +  "\n"+ str(res_f1_macro) + "\n");


    df['newlabel'] = df['teks'].apply(lambda x: negation(x,banyak_label))
    f.write("neg" + "\n")

    res_acc = accuration(df)
    print(res_acc)
    print("===================================================================")
    #res_f1 = fone(df)
    #print(res_f1)
    #print("===================================================================")
    res_f1_macro = fone_macro(df)
    print(res_f1_macro)
    print("===================================================================")
    lst_acc_neg.append(res_acc)
    lst_f1_neg.append(res_f1_macro)
    f.write("acc" + "\n"+ str(res_acc) + "\n" + "f1_macro" +  "\n"+ str(res_f1_macro) + "\n");

print("================================================")
print("Overall Accuracy rep= {} ({})".format(sum(lst_acc_rep)/len(lst_acc_rep), np.std(lst_acc_rep)))
print("Overall F1-Score rep= {} ({})".format(sum(lst_f1_rep)/len(lst_f1_rep), np.std(lst_f1_rep)))
print("================================================")
print("Overall Accuracy adj= {} ({})".format(sum(lst_acc_adj)/len(lst_acc_adj), np.std(lst_acc_adj)))
print("Overall F1-Score adj= {} ({})".format(sum(lst_f1_adj)/len(lst_f1_adj), np.std(lst_f1_adj)))
print("================================================")
print("Overall Accuracy neg= {} ({})".format(sum(lst_acc_neg)/len(lst_acc_neg), np.std(lst_acc_neg)))
print("Overall F1-Score neg= {} ({})".format(sum(lst_f1_neg)/len(lst_f1_neg), np.std(lst_f1_neg)))






templated_posneg_dataset_komentar_instagram_cyberbullying.csv
posneg
acc 200
counter 397
0.5037783375314862
0.3891115016598321
acc 177
counter 397
0.44584382871536526
0.4392655367231638
acc 193
counter 397
0.48614609571788414
0.40189649060616806




templated_posneg_dataset_tweet_sentiment_cellular_service_provider.csv
posneg
acc 167
counter 296
0.5641891891891891
0.4928746928746928
acc 114
counter 296
0.38513513513513514
0.36707552046618735
acc 157
counter 296
0.5304054054054054
0.4712587547388035




templated_posneg_dataset_tweet_sentimen_tayangan_tv.csv
posneg
acc 236
counter 400
0.59
0.5229090909090909
acc 198
counter 400
0.495
0.4598786063798497
acc 215
counter 400
0.5375
0.4856503395402826




templated_posneg_Dataset_C_HT_4000.csv
posneg
acc 2089
counter 4000
0.52225
0.42443221977834317
acc 2045
counter 4000
0.51125
0.473657252198743
acc 2027
counter 4000
0.50675
0.48494619508570014




templated_posneg_dataset_tweet_sentiment_opini_film.csv
posneg
acc 91
counter 198
0.4595

acc 4055
counter 7448
0.5444414607948442
0.4701436860759309




templated_posneg_hotel.csv
posneg
acc 5015
counter 9985
0.5022533800701051
0.3432542921731073
acc 4672
counter 9985
0.46790185277916874
0.4678976685536281
acc 4019
counter 9985
0.4025037556334502
0.36085080426318417
Overall Accuracy rep= 0.5115412588830062 (0.05991626061541504)
Overall F1-Score rep= 0.39738595603311627 (0.08118876624363994)
Overall Accuracy adj= 0.4563077160816603 (0.05206048723496119)
Overall F1-Score adj= 0.4180403710384028 (0.07184944756175844)
Overall Accuracy neg= 0.49072252699945884 (0.052691088074331635)
Overall F1-Score neg= 0.41698957208588094 (0.06384422558129794)
